In [1]:
# importações
import pandas as pd
from sqlalchemy import create_engine, inspect, text

In [2]:
# leitura do arquivo CSV
dados = pd.read_csv("../data/cronologia-marvel-original.csv")
dados.head()

,titulo
0,1943-1945 – Capitão América: O Primeiro Vingad...
1,1995 – Capitã Marvel (Lançado em 2019);
2,2010 – Homem de Ferro (Lançado em 2008);
3,2011 – O Incrível Hulk (Lançado em 2008);
4,2011 – Homem de Ferro 2 (Lançado em 2010);


In [3]:
# extração do ano cronológico
# O ano cronológico é o primeiro número de 4 dígitos encontrado no titulo
dados['ano_cronologico'] = dados.titulo.str.extract(r'(^\d{4})').astype(int)
dados.head()

,titulo,ano_cronologico
0,1943-1945 – Capitão América: O Primeiro Vingad...,1943
1,1995 – Capitã Marvel (Lançado em 2019);,1995
2,2010 – Homem de Ferro (Lançado em 2008);,2010
3,2011 – O Incrível Hulk (Lançado em 2008);,2011
4,2011 – Homem de Ferro 2 (Lançado em 2010);,2011


In [4]:
# remoção do ano cronológico do título
dados.titulo = dados.titulo.str.replace(r'^\d{4}(-\d{4})?\s?–\s?', '', regex=True).str.strip()
dados.head()

,titulo,ano_cronologico
0,Capitão América: O Primeiro Vingador (Lançado ...,1943
1,Capitã Marvel (Lançado em 2019);,1995
2,Homem de Ferro (Lançado em 2008);,2010
3,O Incrível Hulk (Lançado em 2008);,2011
4,Homem de Ferro 2 (Lançado em 2010);,2011


In [5]:
# extração do ano de lançamento
dados['ano_lancamento'] = dados.titulo.str.extract(r'(\d{4}\);)')
dados.head()

,titulo,ano_cronologico,ano_lancamento
0,Capitão América: O Primeiro Vingador (Lançado ...,1943,2011);
1,Capitã Marvel (Lançado em 2019);,1995,2019);
2,Homem de Ferro (Lançado em 2008);,2010,2008);
3,O Incrível Hulk (Lançado em 2008);,2011,2008);
4,Homem de Ferro 2 (Lançado em 2010);,2011,2010);


In [6]:
# normalizando data de lançamento
dados.ano_lancamento = dados.ano_lancamento.str.replace(');', '').astype(int)
dados.head()

,titulo,ano_cronologico,ano_lancamento
0,Capitão América: O Primeiro Vingador (Lançado ...,1943,2011
1,Capitã Marvel (Lançado em 2019);,1995,2019
2,Homem de Ferro (Lançado em 2008);,2010,2008
3,O Incrível Hulk (Lançado em 2008);,2011,2008
4,Homem de Ferro 2 (Lançado em 2010);,2011,2010


In [7]:
# remoção do ano de lançamento do título
dados.titulo = dados.titulo.str.replace(r'\((\S| )+\);', '', regex=True).str.strip()
dados.head()

,titulo,ano_cronologico,ano_lancamento
0,Capitão América: O Primeiro Vingador,1943,2011
1,Capitã Marvel,1995,2019
2,Homem de Ferro,2010,2008
3,O Incrível Hulk,2011,2008
4,Homem de Ferro 2,2011,2010


In [8]:
# booleano se já foi assistido ou não
# O valor padrão é False (não assistido)
dados['assistido'] = False
dados.head()

,titulo,ano_cronologico,ano_lancamento,assistido
0,Capitão América: O Primeiro Vingador,1943,2011,False
1,Capitã Marvel,1995,2019,False
2,Homem de Ferro,2010,2008,False
3,O Incrível Hulk,2011,2008,False
4,Homem de Ferro 2,2011,2010,False


In [9]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titulo           49 non-null     object
 1   ano_cronologico  49 non-null     int64 
 2   ano_lancamento   49 non-null     int64 
 3   assistido        49 non-null     bool  
dtypes: bool(1), int64(2), object(1)
memory usage: 1.3+ KB


In [10]:
# criando engine do sqlite
engine = create_engine('sqlite:///../data/cronologia.db')
dados.to_sql('cronologia', engine, if_exists='replace', index=False)

49

In [11]:
inspector = inspect(engine)
inspector.get_table_names()

['cronologia']

In [12]:
# verificando se os dados foram criados corretamente
query = 'SELECT * FROM cronologia'

with engine.connect() as conexao:
  consulta = conexao.execute(text(query))
  dados = consulta.fetchall()

dados_from_sql = pd.DataFrame(dados, columns = consulta.keys())
dados_from_sql

,titulo,ano_cronologico,ano_lancamento,assistido
0,Capitão América: O Primeiro Vingador,1943,2011,0
1,Capitã Marvel,1995,2019,0
2,Homem de Ferro,2010,2008,0
3,O Incrível Hulk,2011,2008,0
4,Homem de Ferro 2,2011,2010,0
5,Thor,2011,2011,0
6,Os Vingadores,2012,2012,0
7,Homem de Ferro 3,2012,2013,0
8,Thor: O Mundo Sombrio,2013,2013,0
9,Capitão América 2 – O Soldado Invernal,2014,2014,0


In [ ]:
# verificando se os dados foram criados corretamente
dados_from_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titulo           49 non-null     object
 1   ano_cronologico  49 non-null     int64 
 2   ano_lancamento   49 non-null     int64 
 3   assistido        49 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.7+ KB
